In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

Build a chain that takes an article or paragraph and returns a structured JSON with:

title: a short title for the paragraph

summary: 2-sentence summary

keywords: list of 3 keywords

In [77]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers.fix import OutputFixingParser
from langchain_groq import ChatGroq
import pydantic

In [78]:
# define parser data model

class Summary(pydantic.BaseModel):

    title: str
    summary: str
    keywords: list[str]

In [80]:
# define output parser

parser = PydanticOutputParser(pydantic_object=Summary)

In [81]:
# define chat promt template

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI that extracts title, summary and keywords. {format_instructions}"),
        ("user", "{input}")
        ]
    ).partial(format_instructions=parser.get_format_instructions())


In [82]:
# define llm

llm = ChatGroq(model='qwen-qwq-32b')

In [88]:
# define output fix parser

fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

In [ ]:
# connect everything in a chain
chain = prompt | llm | fixing_parser

In [91]:
input_text = """
👋 Hi, I’m Monish Bangera — a data engineer passionate about building intelligent, scalable systems that drive real-world impact.
🚀 I specialize in full-stack data engineering and cloud-native architectures. From designing lakehouse systems on AWS + Snowflake to shipping production-grade APIs, I bring ideas to life with clean, tested, production-ready code.
🧠 Currently diving deep into LangChain, agentic AI, and orchestration frameworks — on the path to mastering AI and building tools to help developers harness modern AI workflows.
📜 I’m AWS Cloud Practitioner and Terraform certified, preparing for the AWS Solutions Architect Associate (SAA) and AWS AI Certification to level up my cloud and AI expertise.
🛠️ Tech I love: Python, Pydantic, Airflow, AWS (S3, Lambda, SNS, MWAA), Snowflake, MongoDB, Terraform, LangChain, Dash, Flask, SQL, GitLab CI/CD
💼 Currently a senior data engineer, leading projects in credit evaluation, billing automation, and NLP pipelines — used by 30+ analysts across teams.
📈 I share my journey, projects, and learnings via content and collaborations — always open to connect and grow in AI + data.
"""

result = chain.invoke(
    {
        "input": input_text,
        }
    )

In [94]:
print(result)

title='Monish Bangera | Senior Data Engineer & Cloud Architect' summary='Monish Bangera is a senior data engineer with expertise in building intelligent, scalable systems through full-stack data engineering and cloud-native architectures. Specializing in AWS and Snowflake lakehouse systems, he designs production-grade APIs and NLP pipelines while focusing on modern AI workflows with LangChain and orchestration frameworks. Certified in AWS and Terraform, he leads projects in credit evaluation, billing automation, and cloud infrastructure, leveraging Python, Airflow, and DevOps tools. Actively pursuing AWS Solutions Architect and AI certifications, Monish shares his technical journey through content and collaborations.' keywords=['Data Engineering', 'Cloud-Native Architectures', 'AWS', 'Snowflake', 'LangChain', 'AI Workflows', 'AWS Cloud Practitioner', 'Terraform Certified', 'AWS SAA', 'AWS AI Certification', 'Python', 'Pydantic', 'Apache Airflow', 'S3', 'Lambda', 'SNS', 'MWAA', 'MongoDB

## ✅ Exercise 2
**Task:** List 5 skills required for an AI Engineer.

In [96]:
# pydantic class for skills

class Ai(pydantic.BaseModel):

    skills: list[str]

In [100]:
# parser

parser = PydanticOutputParser(pydantic_object=Ai)


In [101]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "List 5 skills required for a AI Engineer. {formated_instructions}"),
        ("user", "Please list them")
        ]
    ).partial(formated_instructions=parser.get_format_instructions())



In [105]:
fixing_parser = OutputFixingParser.from_llm(llm=llm, parser=parser)

In [106]:
chain = prompt | llm | fixing_parser

In [108]:
result = chain.invoke({})

In [110]:
print(result)

skills=['Proficiency in programming languages such as Python', 'Strong understanding of machine learning and deep learning algorithms', 'Data preprocessing and analysis skills', 'Experience with AI frameworks (e.g., TensorFlow, PyTorch)', 'Ability to design and implement end-to-end AI solutions']
